In [ ]:
%autosave 0

# MCPC rehearsal problem Oct 25 2017 at UCSY

## Problem E: Stacking Plates

### Input format

- 1st Line: 1 integer, Number of Test Case, each Test Case has following data
   + 1 Line: 1 integer, **n**(Number of Stacks)
   + **n** Lines: first integer: **h** (Number of Plates), and **h** integers (Plate size)
   

### Output format

Case: (test case number): Number of Operations


### Sample Input

```
3
2
3 1 2 4
2 3 5
3
4 1 1 1 1
4 1 1 1 1
4 1 1 1 1
2
15 1 1 1 1 1 2 2 2 2 2 3 3 3 3 3
15 1 1 1 1 1 2 2 2 2 2 3 3 3 3 3
```


### Sample Output

```
Case 1:5
Case 2:2
Case 3:5
```

### Explanation of sample I/O

- 3 test cases
    + Stack of (1,2,4) and (3,5)
    + Stack of 3 (1,1,1,1)
    + Stack of 2 (1,1,1,1,1,2,2,2,2,2,3,3,3,3,3)
    
- 1st case:
Split between 2 and 4, 3 and 5, Move 4 on 5, 3 on 4, (1,2) on 3  ==> Total 5 operations

- 2nd case:
Move 1st stack (1,1,1,1,1) on 2nd stack, move (1st+2nd stack) on 3rd stack  ==> Total 2 operations

- 3rd case:
Split between 1 and 2 of 1st stack, between 2 and 3 of 2nd stack, move (2,2,2,2,2,3,3,3,3,3) of 1st stack on (3,3,3,3,3) of 2nd, move (1,1,1,1,1,2,2,2,2,2) of 2nd stack on it, move (1,1,1,1,1) on top  ==> Total 5 operations

### Specific vs Abstract,  Find a General Rule from Detail

When solving problems and puzzles that you can not find how to solve, thinking specificlly then abstractly is important. Finding general rules from specific examples give you a path to the answer.

- Think about simple cases
- Find general pattern (idea, rule) from there
- Proove the rule (if possible)
- Extend the rule to more complex cases

### How to calculate number of operation(movement)

If there are N stacks and each contains just 1 piece (Split is not necessary), (N-1) operations are reuired. (N-1) is a minimum number of oprations.

For each Split operation, Join operation is required to create single stack. Total number of operation increases by 2 for each Split operations (S). The order of Split and Join does not affect total number of movement. (Split-Split-Join-Join) = (Split-Join-Split-Join) \begin{equation} Nmber Of Movement = 2S + (N-1) \end{equation}

Same size of pieces in original stacks (Case 2 and Case 3) can be considered to be same as single piece. Case 2: 3 stack of (1), Case 3: 2 stack of (1,2,3)

### Optimized movement

Reverse-Thinking is sometimes very effective. Create Final Stack and check the boundary. If the combination of the boundary exists in original stacks, it can be used (not necessary to split). **Stack ID needs to be checked, as for detail see later**.

$S = (Maximum Number Of Split) - (Number Of Reused Boundary)$

- Case 1: [1,2,3,4,5] is final form. Boundary of [1,2] exist in Original Stack-1, $S=(2+1)-1=2, Movement=2*2+(2-1)=5$
- Case 2: Convert original stacks to [1], $Movement=2*0+(3-1)=2$
- Case 3: Convert original stacks to [1,2,3], Final form is [1,1,2,2,3,3]. Boundary of [1,2] and [2,3] exists. $S=(2+2)-2=2$

### Sample I/O gives hint

Sample Input/Output often gives great hint to solve problems. Same number in original stack cause problem in above idea, but same number can be considered to be 1 digit, so convert input data to eliminate duplicate number.

### Stack ID checking

- Assign stack ID
- Merge every plates and sort by radius (size of plate)
- Manage the list of candidate for boundary reuse (top and bottom)
- Boundary assignment can be done greedy, if there is only 1 combination between top of stack and next, use it


In [1]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

'''
    2017 MCPC at UCSY
    Problem-E: Stacking Plates
'''
import sys


class TestCase():
    pass


class Plates():
    # Groupt of same radius plates in merged stack
    # id_list: list of stack ID
    def __init__(self, radius, id_list):
        self.radius = radius
        self.id_list = id_list
        self.top = None
        self.bottom = None

    def match_prev(self, prev_bottom):
        self.top = list()
        for stack_id in self.id_list:
            if stack_id in prev_bottom:
                self.top.append(stack_id)
        self.bottom = self.id_list.copy()
        if len(self.top) == 1 and len(self.bottom) != 1:
            self.bottom.remove(self.top[0])

        return

    def __repr__(self):
        return ('Plates {}: {}, top: {} bottom: {}'.format(self.radius, self.id_list, self.top, self.bottom))


def parse_tc(tc):
    '''
        Input: Test Case
        Update: 
        Return: None
    '''

    tc.n = int(tc.stdin.readline())
    tc.stacks = list()

    for i in range(tc.n):
        stack = tc.stdin.readline().split()[1:]  # 2d List, 1st=len
        tc.stacks.append(stack)

    return


def reform_stack(org):
    '''
        Input: tc.stacks
        Output: cosolidated stacks (no prefix, no duplicate)
    '''

    stacks = list()
    stack_id = 0

    for stack in org:
        prev_radius = None
        new_stack = list()

        for radius in stack:
            if radius != prev_radius:
                new_stack.append((radius, stack_id))
            prev_radius = radius

        stacks.append(new_stack)
        stack_id += 1

    return stacks


def merge(stacks):
    '''
        stacks: 2D List of tuple(radius, id)
        Return: 1D sorted List
    '''

    merged_stack = list()

    for stack in stacks:
        merged_stack.extend(stack)

    merged_stack.sort()

    return merged_stack


def stack2plates(merged_stack):
    '''
    merged_stack: List of Tuple(radius, id)
    return: List of Plates
    '''

    plates_list = list()
    id_list = list()
    prev_size = None

    for plate in merged_stack:
        radius, plate_id = plate
        if radius != prev_size:
            if id_list:
                plates_list.append(Plates(prev_size, id_list))
            id_list = [plate_id]
        else:
            id_list.append(plate_id)

        prev_size = radius

    if id_list:
        plates_list.append(Plates(radius, id_list))

    return plates_list


def max_reuse(plates_list):

    reuse = 0
    prev_bottom = list()

    for plates in plates_list:
        plates.match_prev(prev_bottom)
        if plates.top: reuse += 1
        prev_bottom = plates.bottom
        #print(plates, file=sys.stderr)

    return reuse

def solve(tc):
    '''
        Input: Test Case
        Return: Numper of movement
    '''

    parse_tc(tc)
    stacks = reform_stack(tc.stacks)
    #print(stacks)
    num_merge = len(stacks) - 1  ## Join Stacks
    for stack in stacks:
        num_merge += (len(stack) - 1) * 2  ## Split and Join

    merged_stack = merge(stacks)
    plates_list = stack2plates(merged_stack)  # list of Plates

    #return (num_merge - check_bound(merged_stack, stack_bound) * 2)
    return (num_merge - max_reuse(plates_list) * 2)


In [4]:
### Main routine

infile = open('reh_e.in', 'r')

tc = TestCase()
tc.stdin = infile
tc.t = int(tc.stdin.readline())

for i in range(tc.t):
    print('Case {}:{}'.format(i+1, solve(tc)))

Case 1:5
Case 2:2
Case 3:5
Case 4:8
Case 5:3
